In [ ]:
!pip install torchvision torch pytorch-grad-cam


ERROR: Could not find a version that satisfies the requirement pytorch-grad-cam (from versions: none)
ERROR: No matching distribution found for pytorch-grad-cam


In [ ]:
!pip install grad-cam

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from pytorch_grad_cam import GradCAM # this should now work
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Path to your dataset (replace with your actual path in Google Drive)
img_folder = '/content/drive/MyDrive/pocs/'
infected_folder = img_folder + 'infected/'
non_infected_folder = img_folder + 'notinfected/' #Fixed typo: changed 'non_infected' to 'non-infected'


In [ ]:
import os
img_paths = []
labels = []

In [ ]:
# Assuming you have two folders 'infected' and 'non_infected'
# Collect infected images
for img_name in os.listdir(infected_folder):
    img_paths.append(os.path.join(infected_folder, img_name))
    labels.append(1)  # Label 1 for infected

In [ ]:
# Collect non-infected images
for img_name in os.listdir(non_infected_folder):
    img_paths.append(os.path.join(non_infected_folder, img_name))
    labels.append(0)  # Label 0 for non-infected

In [ ]:
# Define the MedicalDataset class
class MedicalDataset(Dataset):
    def __init__(self, img_paths, labels, transform=None):
        self.img_paths = img_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('RGB') # Ensure the image is loaded as RGB
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [ ]:
# Create dataset and dataloader
dataset = MedicalDataset(img_paths, labels, transform=transform)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)


In [ ]:

# Load DenseNet201 pre-trained model
model = models.densenet201(pretrained=True)

# Replace the classifier layer to fit your number of classes
model.classifier = nn.Linear(1920, 2)  # Assuming binary classification (infected vs non-infected)

# Move the model to the appropriate device (GPU/CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet201-c1103571.pth" to /root/.cache/torch/hub/checkpoints/densenet201-c1103571.pth
100%|██████████| 77.4M/77.4M [00:00<00:00, 121MB/s]


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs =25  # Adjust based on your dataset

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in data_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(data_loader)}')


Epoch [1/25], Loss: 0.010713125930469837


In [ ]:
# Apply Grad-CAM
target_layers = [model.features.denseblock4]  # Select the target layer to visualize

# Remove use_cuda argument
cam = GradCAM(model=model, target_layers=target_layers)

In [ ]:
# Test the model on a new image
image_path = '/content/drive/MyDrive/pocs/infected/1 (8).jpg'  # Your test image
img = Image.open(image_path).convert('RGB')
input_tensor = transform(img).unsqueeze(0).to(device)



In [ ]:
# Generate CAM
targets = [ClassifierOutputTarget(1)]  # Assuming label 1 corresponds to 'infected'
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)


In [ ]:
# Resize the original image to match the CAM dimensions
img = img.resize((224, 224))

# Convert the image to the correct data type and number of channels
img_np = np.array(img)
if len(img_np.shape) == 2:  # Grayscale image
    img_np = cv2.cvtColor(img_np, cv2.COLOR_GRAY2BGR)

# Normalize the image to be in the range [0, 255] and ensure it's uint8
img_np = np.uint8(255 * img_np / np.max(img_np))

# Ensure grayscale_cam is in the correct format (CV_8UC1)
if len(grayscale_cam.shape) == 3:
    grayscale_cam = grayscale_cam[:, :, 0] #If grayscale_cam has more than one channel, take the first one.
grayscale_cam = np.uint8(255 * grayscale_cam)

# Generate CAM
cam_image = show_cam_on_image(img_np, grayscale_cam, use_rgb=True)

In [ ]:
# Display the result
plt.imshow(cam_image)
plt.show()

In [ ]:
# 8. Display the original image and the Grad-CAM overlay
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np) # Changed image_np to img_np

plt.subplot(1, 2, 2)
plt.title('Grad-CAM')
plt.imshow(cam_image)

plt.show()

#Applying on diffrent data sample _ Infected

In [ ]:
# Test the model on a new image
image_path = '/content/drive/MyDrive/pocs/infected/1 (13).jpg'  # Your test image
img = Image.open(image_path).convert('RGB')
input_tensor = transform(img).unsqueeze(0).to(device)

In [ ]:

# Generate CAM
targets = [ClassifierOutputTarget(1)]  # Assuming label 1 corresponds to 'infected'
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

# Resize the original image to match the CAM dimensions
img = img.resize((224, 224))
img_np = np.array(img) / 255.0

# Ensure grayscale_cam has the correct number of channels
if len(grayscale_cam.shape) == 2:
    grayscale_cam = grayscale_cam[..., np.newaxis]

In [ ]:
# Generate CAM
cam_image = show_cam_on_image(img_np, grayscale_cam[0, :], use_rgb=True)

# 8. Display the original image and the Grad-CAM overlay
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np)

plt.subplot(1, 2, 2)
plt.title('Grad-CAM')
plt.imshow(cam_image)

plt.show()

#Applying on diffrent data sample _ Infected

In [ ]:
# Test the model on a new image
image_path = '/content/drive/MyDrive/pocs/infected/1 (28).jpg'  # Your test image
img = Image.open(image_path).convert('RGB')
input_tensor = transform(img).unsqueeze(0).to(device)

# Generate CAM
targets = [ClassifierOutputTarget(1)]  # Assuming label 1 corresponds to 'infected'
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

# Resize the original image to match the CAM dimensions
img = img.resize((224, 224))
img_np = np.array(img) / 255.0

# Ensure grayscale_cam has the correct number of channels
if len(grayscale_cam.shape) == 2:
    grayscale_cam = grayscale_cam[..., np.newaxis]

# Generate CAM
cam_image = show_cam_on_image(img_np, grayscale_cam[0, :], use_rgb=True)

# 8. Display the original image and the Grad-CAM overlay
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np)

plt.subplot(1, 2, 2)
plt.title('Grad-CAM')
plt.imshow(cam_image)

plt.show()

#ScoreCAM


In [ ]:
!pip install pytorch-gradcam

from pytorch_grad_cam import ScoreCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# ... your existing code for model loading, transform, and device ...

# Test the model on a new image
image_path = '/content/drive/MyDrive/pocs/infected/1 (28).jpg'  # Your test image
img = Image.open(image_path).convert('RGB')
input_tensor = transform(img).unsqueeze(0).to(device)

# Initialize Score-CAM
cam = ScoreCAM(model=model, target_layers=target_layers) # Replace model and target_layers

# Generate CAM
targets = [ClassifierOutputTarget(1)]  # Assuming label 1 corresponds to 'infected'
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

# Resize the original image to match the CAM dimensions
img = img.resize((224, 224))
img_np = np.array(img) / 255.0

# Ensure grayscale_cam has the correct number of channels
if len(grayscale_cam.shape) == 2:
    grayscale_cam = grayscale_cam[..., np.newaxis]

# Generate CAM
cam_image = show_cam_on_image(img_np, grayscale_cam[0, :], use_rgb=True)

# Display the original image and the Score-CAM overlay
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np)
plt.subplot(1, 2, 2)
plt.title('Score-CAM')
plt.imshow(cam_image)
plt.show()

#GradCAMPlusPlus

In [ ]:
!pip install pytorch-gradcam

from pytorch_grad_cam import GradCAMPlusPlus
from pytorch_grad_cam.utils.image import show_cam_on_image
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# ... your existing code for model loading, transform, and device ...

# Test the model on a new image
image_path = '/content/drive/MyDrive/pocs/infected/1 (28).jpg'
img = Image.open(image_path).convert('RGB')
input_tensor = transform(img).unsqueeze(0).to(device)

# Initialize GradCAM++
cam = GradCAMPlusPlus(model=model, target_layers=target_layers)

# Generate CAM
targets = [ClassifierOutputTarget(1)]
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

# ... (Resize, handle channels, and visualize as in the previous example) ...
# Display the original
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np)
plt.subplot(1, 2, 2)
plt.title('Gradcam++')
plt.imshow(cam_image)
plt.show()

#XGradCAM

In [ ]:
from pytorch_grad_cam import XGradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget

# Initialize XGrad-CAM
cam = XGradCAM(model=model, target_layers=target_layers)

# Generate the heatmap
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

# Visualize
cam_image = show_cam_on_image(img_np, grayscale_cam[0], use_rgb=True)
plt.imshow(cam_image)
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np)
plt.subplot(1, 2, 2)
plt.title('XGradCAM')
plt.imshow(cam_image)
plt.show()


#EigenCAM

In [ ]:
from pytorch_grad_cam import EigenCAM

# Initialize EigenCAM
cam = EigenCAM(model=model, target_layers=target_layers)

# Generate the heatmap
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

# Visualize
cam_image = show_cam_on_image(img_np, grayscale_cam[0], use_rgb=True)
plt.imshow(cam_image)
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np)
plt.subplot(1, 2, 2)
plt.title('EigenCAM')
plt.imshow(cam_image)
plt.show()


#LayerCAM

In [ ]:
from pytorch_grad_cam import LayerCAM

# Initialize LayerCAM
cam = LayerCAM(model=model, target_layers=target_layers)

# Generate the heatmap
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

# Visualize
cam_image = show_cam_on_image(img_np, grayscale_cam[0], use_rgb=True)
plt.imshow(cam_image)
plt.show()

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np)
plt.subplot(1, 2, 2)
plt.title('LayerCAM')
plt.imshow(cam_image)
plt.show()

#Full-Gradient /  FullGrad

In [ ]:
from pytorch_grad_cam import FullGrad

# Initialize Full-Gradient with target layers
cam = FullGrad(model=model, target_layers=target_layers)

# Generate the heatmap
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

# Visualize
cam_image = show_cam_on_image(img_np, grayscale_cam[0], use_rgb=True)
plt.imshow(cam_image)
plt.show()


plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np)
plt.subplot(1, 2, 2)
plt.title('Full-Gradient')
plt.imshow(cam_image)
plt.show()

#Thresholded CAM (Binary Mask)
You can apply a threshold to the heatmap after generating it, converting the values into a binary mask (0/1). This highlights only the most activated regions.

In [ ]:
# Generate CAM (using any method, e.g., AblationCAM, GradCAM)
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

# Apply a threshold to binarize the heatmap (e.g., threshold at 0.5)
threshold = 0.5
binary_cam = (grayscale_cam[0] > threshold).astype(float)

# Visualize the binary heatmap
cam_image = show_cam_on_image(img_np, binary_cam, use_rgb=True)
plt.imshow(cam_image)
plt.show()

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np)
plt.subplot(1, 2, 2)
plt.title('Thresholded CAM ')
plt.imshow(cam_image)
plt.show()


In [ ]:
from lime import lime_image
from skimage.segmentation import mark_boundaries
import torch
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# Load and preprocess the image
image_path = '/content/drive/MyDrive/pocs/infected/1 (13).jpg'
img = Image.open(image_path).convert('RGB')
img = img.resize((224, 224))  # Assuming your model expects 224x224 input
img_np = np.array(img)

# Convert the image to a format the model can accept
input_tensor = transform(img).unsqueeze(0).to(device)

# Define a prediction function that returns model output probabilities
def predict_fn(images):
    # Convert images to tensors and pass through the model
    inputs = [transform(Image.fromarray(image)).unsqueeze(0).to(device) for image in images]
    inputs = torch.cat(inputs, dim=0)
    with torch.no_grad():
        outputs = model(inputs)  # Assuming your model variable is named `model`
    return outputs.cpu().numpy()

# Initialize LIME explainer
explainer = lime_image.LimeImageExplainer()

# Get LIME explanation
explanation = explainer.explain_instance(img_np,
                                         classifier_fn=predict_fn,
                                         top_labels=2,
                                         hide_color=0,
                                         num_samples=1000)

# Show explanation for class label 1 (infected)
temp, mask = explanation.get_image_and_mask(label=1, positive_only=False, num_features=10, hide_rest=False)

# Display the LIME result
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np)

plt.subplot(1, 2, 2)
plt.title('LIME Explanation (Infected)')
plt.imshow(mark_boundaries(temp, mask))

plt.show()


OcclusionSensitivity

#2. SHAP (SHapley Additive exPlanations)
SHAP values explain the output of a machine learning model by showing the contribution of each feature.

In [ ]:
import shap
import numpy as np
import torch
from PIL import Image

# Preprocessing the image
image_path = '/content/drive/MyDrive/pocs/infected/1 (13).jpg'
img = Image.open(image_path).convert('RGB')
img = img.resize((224, 224))
img_np = np.array(img) / 255.0
input_tensor = transform(img).unsqueeze(0).to(device)

# SHAP Explainer
explainer = shap.DeepExplainer(model, input_tensor)
shap_values = explainer.shap_values(input_tensor)

# Visualize SHAP explanation
shap.image_plot(shap_values, np.array([img_np]))


In [ ]:
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Load and preprocess the image
image_path = '/content/drive/MyDrive/pocs/infected/1 (13).jpg'
img = Image.open(image_path).convert('RGB')
img = img.resize((224, 224))
img_np = np.array(img) / 255.0
input_tensor = transform(img).unsqueeze(0).to(device)

# Function to apply occlusion
def apply_occlusion(image, size=32, stride=8):
    heatmap = np.zeros((image.shape[1], image.shape[2]))
    for y in range(0, image.shape[1] - size + 1, stride):
        for x in range(0, image.shape[2] - size + 1, stride):
            occluded_image = image.clone()
            occluded_image[:, :, y:y+size, x:x+size] = 0  # Mask out a patch
            output = model(occluded_image).squeeze()
            heatmap[y:y+size, x:x+size] = output[1].item()  # Assuming label 1 is 'infected'
    return heatmap

# Generate occlusion heatmap
occlusion_heatmap = apply_occlusion(input_tensor)

# Visualize the occlusion sensitivity map
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np)

plt.subplot(1, 2, 2)
plt.title('Occlusion Sensitivity')
plt.imshow(occlusion_heatmap, cmap='jet')
plt.show()


In [ ]:
import torch
from captum.attr import IntegratedGradients
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Load and preprocess the image
image_path = '/content/drive/MyDrive/pocs/infected/1 (13).jpg'
img = Image.open(image_path).convert('RGB')
img = img.resize((224, 224))
img_np = np.array(img) / 255.0
input_tensor = transform(img).unsqueeze(0).to(device)

# Integrated Gradients explainer
ig = IntegratedGradients(model)
attributions = ig.attribute(input_tensor, target=1)  # Assuming label 1 is 'infected'
attributions = attributions.squeeze().cpu().detach().numpy()

# Normalize attributions for visualization
attributions = np.sum(np.abs(attributions), axis=0)
attributions = (attributions - np.min(attributions)) / (np.max(attributions) - np.min(attributions))

# Plot Integrated Gradients
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np)

plt.subplot(1, 2, 2)
plt.title('Integrated Gradients')
plt.imshow(attributions, cmap='hot')
plt.show()


In [ ]:
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Function to add noise to the image
def add_noise(image, noise_level=0.1):
    noise = noise_level * torch.randn_like(image).to(device)
    return image + noise

# Function to compute gradients with noise (SmoothGrad)
def smoothgrad(input_tensor, n_samples=50, noise_level=0.1):
    smoothed_grad = torch.zeros_like(input_tensor)
    for i in range(n_samples):
        noisy_input = add_noise(input_tensor, noise_level)
        noisy_input.requires_grad = True
        output = model(noisy_input).squeeze()[1]  # Assuming label 1 is 'infected'
        output.backward()
        smoothed_grad += noisy_input.grad.abs()
    return smoothed_grad / n_samples

# Load and preprocess the image
image_path = '/content/drive/MyDrive/pocs/infected/1 (13).jpg'
img = Image.open(image_path).convert('RGB')
img = img.resize((224, 224))
img_np = np.array(img) / 255.0
input_tensor = transform(img).unsqueeze(0).to(device)

# Generate SmoothGrad explanation
smoothed_grad = smoothgrad(input_tensor).squeeze().cpu().detach().numpy()

# Normalize for visualization
smoothed_grad = np.sum(np.abs(smoothed_grad), axis=0)
smoothed_grad = (smoothed_grad - np.min(smoothed_grad)) / (np.max(smoothed_grad) - np.min(smoothed_grad))

# Plot SmoothGrad explanation
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np)

plt.subplot(1, 2, 2)
plt.title('SmoothGrad')
plt.imshow(smoothed_grad, cmap='hot')
plt.show()


Summary
LIME: Perturbs the image to identify key features affecting the prediction.
SHAP: Uses game theory to identify the contribution of individual features.
Occlusion Sensitivity: Masks parts of the image to determine the importance of different regions.
Integrated Gradients: Averages gradients over interpolated inputs for smoother attributions.
SmoothGrad: Adds noise to inputs and averages gradients for smoother visualizations.

Here are some more latest XAI techniques beyond the traditional ones like GradCAM, LIME, and Integrated Gradients. These newer methods often provide more nuanced or application-specific insights into model behavior.

1. Captum's Guided Backpropagation
Guided Backpropagation modifies the standard backpropagation algorithm to suppress negative gradients, producing cleaner and sharper feature visualizations.

In [ ]:
from captum.attr import GuidedBackprop
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Load and preprocess the image
image_path = '/content/drive/MyDrive/pocs/infected/1 (13).jpg'
img = Image.open(image_path).convert('RGB')
img = img.resize((224, 224))
img_np = np.array(img) / 255.0
input_tensor = transform(img).unsqueeze(0).to(device)

# Guided Backpropagation explainer
guided_bp = GuidedBackprop(model)
attributions = guided_bp.attribute(input_tensor, target=1)  # Assuming label 1 is 'infected'

# Convert attributions to numpy
attributions = attributions.squeeze().cpu().detach().numpy()
attributions = np.sum(attributions, axis=0)  # Summing channels
attributions = np.maximum(attributions, 0)  # ReLU to remove negative values
attributions = (attributions - attributions.min()) / (attributions.max() - attributions.min())  # Normalize

# Visualize Guided Backpropagation
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np)

plt.subplot(1, 2, 2)
plt.title('Guided Backpropagation')
plt.imshow(attributions, cmap='hot')
plt.show()


3. Score-Based Explainability (SBE)
SBE assigns importance scores to input features by modeling how much each feature contributes to the prediction score. Unlike gradient-based methods, SBE computes scores directly from the model’s internal values without requiring backpropagation.

In [ ]:
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Load and preprocess the image
image_path = '/content/drive/MyDrive/pocs/infected/1 (13).jpg'
img = Image.open(image_path).convert('RGB')
img = img.resize((224, 224))
img_np = np.array(img) / 255.0
input_tensor = transform(img).unsqueeze(0).to(device)

# Forward pass and scoring
with torch.no_grad():
    model.eval()
    output = model(input_tensor)
    output_score = output[0, 1].item()  # Get the score for 'infected' label

# Create a simple scoring mechanism based on feature importance
def score_features(input_tensor, score_function):
    scores = np.zeros(input_tensor.shape[-2:])
    for y in range(input_tensor.shape[-2]):
        for x in range(input_tensor.shape[-1]):
            perturbed_input = input_tensor.clone()
            perturbed_input[:, :, y, x] = 0  # Zero out the pixel (perturbation)
            perturbed_output = model(perturbed_input)[0, 1].item()
            scores[y, x] = output_score - perturbed_output  # Score is the difference in output
    return scores

# Compute feature scores using SBE
scores = score_features(input_tensor, output_score)

# Normalize for visualization
scores = (scores - scores.min()) / (scores.max() - scores.min())

# Visualize SBE heatmap
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np)

plt.subplot(1, 2, 2)
plt.title('SBE Heatmap')
plt.imshow(scores, cmap='hot')
plt.show()


In [ ]:
from captum.attr import DeepLift
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Load and preprocess the image
image_path = '/content/drive/MyDrive/pocs/infected/1 (13).jpg'
img = Image.open(image_path).convert('RGB')
img = img.resize((224, 224))
img_np = np.array(img) / 255.0
input_tensor = transform(img).unsqueeze(0).to(device)

# DeepLIFT explainer
deeplift = DeepLift(model)
attributions = deeplift.attribute(input_tensor, target=1)  # Assuming label 1 is 'infected'

# Normalize attributions for visualization
attributions = attributions.squeeze().cpu().detach().numpy()
attributions = np.sum(attributions, axis=0)  # Summing channels
attributions = (attributions - np.min(attributions)) / (np.max(attributions) - np.min(attributions))

# Visualize DeepLIFT attributions
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np)

plt.subplot(1, 2, 2)
plt.title('DeepLIFT Attributions')
plt.imshow(attributions, cmap='hot')
plt.show()


In [ ]:
import torch
import numpy as np
from PIL import Image, ImageFilter
import matplotlib.pyplot as plt

# Load and preprocess the image
image_path = '/content/drive/MyDrive/pocs/infected/1 (13).jpg'
img = Image.open(image_path).convert('RGB')
img = img.resize((224, 224))
img_np = np.array(img) / 255.0
input_tensor = transform(img).unsqueeze(0).to(device)

# Function to apply perturbation (Gaussian Blur)
def apply_perturbation(img_np, patch_size=32, stride=8):
    perturbed_img = img.copy()
    for y in range(0, img_np.shape[0] - patch_size + 1, stride):
        for x in range(0, img_np.shape[1] - patch_size + 1, stride):
            patch = img.crop((x, y, x + patch_size, y + patch_size))
            blurred_patch = patch.filter(ImageFilter.GaussianBlur(radius=5))
            perturbed_img.paste(blurred_patch, (x, y))
    return np.array(perturbed_img) / 255.0

# Generate Perturbation-Based Saliency Map
perturbed_img_np = apply_perturbation(img_np)

# Visualize Perturbation-Based Saliency Map
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np)

plt.subplot(1, 2, 2)
plt.title('Perturbation-Based Saliency Map')
plt.imshow(perturbed_img_np)
plt.show()


Summary of Advanced XAI Techniques:
Guided Backpropagation: Generates sharp feature attributions by modifying backpropagation.
DIME: Disentangles multi-modal data (like text and images) for separate interpretations.
SBE: Directly assigns scores based on how important each feature is to the prediction.
DeepLIFT: Provides faster, more efficient feature attributions than Integrated Gradients.
TCAV: Focuses on testing concepts (like "infected" or "healthy") for their impact on predictions.
PBSM: Introduces perturbations to the image to